In [1]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from config import *
import utils
from tqdm import tqdm
from datetime import datetime
import db_utils
from bson.objectid import ObjectId
import json

In [2]:
client = MongoClient(f"mongodb+srv://{DB_USER}:{DB_PASSWORD}@{DB_URL}/?retryWrites=true&w=majority")

db = client[DB_NAME]

db_utils.init(db)

-- DB initialized -- 


In [3]:
new_rooms_ewa = json.load(open("new_rooms_ewa.json", "r"))

# uppercase all rooms
new_rooms_ewa = [k.upper() for k in new_rooms_ewa]

In [4]:
utils.create_rooms(db, rooms_names=new_rooms_ewa)

Getting rooms from plan.epfl.ch


100%|██████████| 11/11 [00:24<00:00,  2.20s/it]


Parsing rooms...


100%|██████████| 15542/15542 [01:29<00:00, 173.71it/s]


Found 15542 rooms on plan.epfl.ch
Getting rooms from database
Found 170 rooms in database
Updating rooms in database


100%|██████████| 170/170 [00:22<00:00,  7.51it/s]


Filtering rooms to create


100%|██████████| 9/9 [00:00<00:00, 992.84it/s]


Inserting 9 new rooms in database


In [28]:
def fetchRoomPage(room_name):
    URL = f"https://ewa.epfl.ch/room/Default.aspx?room={room_name}"
    
    try :
        r = requests.get(url = URL, timeout=10)
    except requests.exceptions.ConnectionError:
        print('ConnectionError: ', room_name)
        return None
        
    r = requests.get(url = URL, timeout=10)

    soup = BeautifulSoup(r.text, 'html.parser')

    # Find the content placeholder
    content_placeholder = soup.find(id="ContentPlaceHolder1_Label1")

    if content_placeholder is None:
        return None
    elif "Pas d'information pour cette salle..." == content_placeholder.text:
        return False
    else:
        return True

In [6]:
rooms = list(db.rooms.find({
    'available': True
}))

In [29]:
# scan all rooms to see if they exist on ewa
available_rooms = []
for room in tqdm(rooms, total=len(rooms)):
    room_name = room['name']
    if fetchRoomPage(room_name):
        available_rooms.append(room_name)
        print(f"Room {room_name} is available")
    

  1%|          | 1/170 [00:04<13:46,  4.89s/it]

ConnectionError:  MEB331


  2%|▏         | 3/170 [00:17<17:34,  6.31s/it]

Room INF1 is available


  6%|▋         | 11/170 [01:07<10:12,  3.85s/it]

...

Room INF3 is available


 96%|█████████▌| 163/170 [05:21<01:21, 11.68s/it]

Room INJ218 is available


100%|██████████| 170/170 [06:04<00:00,  2.15s/it]

ConnectionError:  GRB001


In [33]:
# export the list of available rooms to a json
with open('ewa_rooms.json', 'w') as f:
    json.dump(available_rooms, f)